# Mercedes F1 Infringement Document Filter

This notebook extracts Mercedes-specific infringement documents from the FIA PDF collection and converts them to text files for further processing.

## Objective
- Parse all PDF files in the Documents folder
- Filter for documents addressed to "Mercedes-AMG PETRONAS F1 Team"
- Extract text content from filtered PDFs
- Save as individual .txt files organized by year


In [2]:
pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Import required libraries
import os
import re
from pathlib import Path
import PyPDF2
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [15]:
# Define paths
base_path = Path("Documents")
years = ["2020-infirdgement_profile", "2021-infridgement_profile", 
         "2022-infridgement_profile", "2023-infridgement_profile"]

print("Available year folders:")
for year in years:
    year_path = base_path / year
    if year_path.exists():
        pdf_count = len(list(year_path.glob("*.pdf")))
        print(f"  {year}: {pdf_count} PDF files")


Available year folders:
  2020-infirdgement_profile: 131 PDF files
  2021-infridgement_profile: 175 PDF files
  2022-infridgement_profile: 227 PDF files
  2023-infridgement_profile: 194 PDF files


In [16]:
# Function to extract text from PDF
def extract_pdf_text(pdf_path):
    """
    Extract text from PDF file
    Returns the full text content
    """
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text() + "\n"
            
            return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None


In [17]:
# Updated function to check if document is addressed to Mercedes (includes car numbers)
def is_mercedes_document_updated(text):
    """
    Check if the document is addressed to Mercedes team or involves Mercedes drivers
    Look for the specific header pattern and car numbers
    """
    if not text:
        return False
    
    # Convert to lowercase for case-insensitive matching
    text_lower = text.lower()
    
    # Look for Mercedes team references in the header section
    mercedes_patterns = [
        r'to:\s*the team manager,?\s*mercedes[-\s]*amg\s*petronas\s*f1\s*team',
        r'mercedes[-\s]*amg\s*petronas\s*f1\s*team',
        r'mercedes\s*petronas\s*f1\s*team',
        r'mercedes\s*amg\s*petronas'
    ]
    
    # Mercedes car numbers by year:
    # 2020-2021: Car 44 (Hamilton), Car 77 (Bottas)
    # 2022-2023: Car 44 (Hamilton), Car 63 (Russell)
    # Also Car 42 (Bottas in some contexts)
    mercedes_car_numbers = [44, 77, 63, 42]
    
    # Check first 2000 characters (header section)
    header_section = text_lower[:2000]
    
    # Check for Mercedes team references
    for pattern in mercedes_patterns:
        if re.search(pattern, header_section, re.IGNORECASE):
            return True
    
    # Check for Mercedes car numbers in document title/header
    # Look for patterns like "Car 44", "Car 77", etc.
    for car_num in mercedes_car_numbers:
        car_pattern = rf'\bcar\s*{car_num}\b'
        if re.search(car_pattern, header_section, re.IGNORECASE):
            return True
    
    return False

# Use the updated function instead of the original
is_mercedes_document = is_mercedes_document_updated


In [18]:
# Function to check if document is addressed to Mercedes
def is_mercedes_document(text):
    """
    Check if the document is addressed to Mercedes team
    Look for the specific header pattern
    """
    if not text:
        return False
    
    # Convert to lowercase for case-insensitive matching
    text_lower = text.lower()
    
    # Look for Mercedes team references in the header section
    mercedes_patterns = [
        r'to:\s*the team manager,?\s*mercedes[-\s]*amg\s*petronas\s*f1\s*team',
        r'mercedes[-\s]*amg\s*petronas\s*f1\s*team',
        r'mercedes\s*petronas\s*f1\s*team',
        r'mercedes\s*amg\s*petronas'
    ]
    
    # Check first 2000 characters (header section)
    header_section = text_lower[:2000]
    
    for pattern in mercedes_patterns:
        if re.search(pattern, header_section, re.IGNORECASE):
            return True
    
    return False


In [19]:
# Function to clean and save text content
def clean_text(text):
    """
    Clean extracted text
    """
    if not text:
        return ""
    
    # Remove excessive whitespace and normalize
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text


In [20]:
# Main processing function
def process_year_folder(year_folder):
    """
    Process all PDFs in a year folder and extract Mercedes documents
    """
    year_path = base_path / year_folder
    if not year_path.exists():
        print(f"Folder {year_path} does not exist")
        return
    
    pdf_files = list(year_path.glob("*.pdf"))
    print(f"\nProcessing {len(pdf_files)} PDF files in {year_folder}...")
    
    mercedes_docs = []
    
    for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
        # Extract text from PDF
        text = extract_pdf_text(pdf_file)
        
        if text and is_mercedes_document(text):
            # Clean the text
            cleaned_text = clean_text(text)
            
            # Create output filename
            txt_filename = pdf_file.stem + ".txt"
            txt_path = year_path / txt_filename
            
            # Save as text file
            try:
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write(cleaned_text)
                
                mercedes_docs.append({
                    'pdf_file': pdf_file.name,
                    'txt_file': txt_filename,
                    'year': year_folder,
                    'text_length': len(cleaned_text)
                })
                
                print(f"✓ Found Mercedes document: {pdf_file.name}")
                
            except Exception as e:
                print(f"Error saving {txt_filename}: {e}")
    
    return mercedes_docs


In [21]:
# Process all year folders
all_mercedes_docs = []

for year_folder in years:
    mercedes_docs = process_year_folder(year_folder)
    if mercedes_docs:
        all_mercedes_docs.extend(mercedes_docs)
        print(f"Found {len(mercedes_docs)} Mercedes documents in {year_folder}")
    else:
        print(f"No Mercedes documents found in {year_folder}")



Processing 131 PDF files in 2020-infirdgement_profile...


Processing PDFs:   5%|▌         | 7/131 [00:00<00:10, 11.42it/s]

✓ Found Mercedes document: 2020 Austrian Grand Prix - Decision - Car 44 - alleged failure to slow for yellow flags.pdf


Processing PDFs:  12%|█▏        | 16/131 [00:01<00:06, 18.63it/s]

✓ Found Mercedes document: 2020 Austrian Grand Prix - Decision - review of decision (document 33).pdf
✓ Found Mercedes document: 2020 Austrian Grand Prix - Offence - Car 44 - Failure to slow for yellow flags (post review).pdf
✓ Found Mercedes document: 2020 Austrian Grand Prix - Offence - Car 44 - incident with car 23.pdf
✓ Found Mercedes document: 2020 Austrian Grand Prix - Offence - Car 44 - Leaving the track in turn 10.pdf
✓ Found Mercedes document: 2020 Austrian Grand Prix - Offence - Car 44 - Track Limits turn 10.pdf


...

✓ Found Mercedes document: 2023 Qatar Grand Prix - Infringement - Car 44 - Crossing the track.pdf
✓ Found Mercedes document: 2023 Saudi Arabian Grand Prix - Offence - Mercedes - Inaccurate Scrutineering Form.pdf


Processing PDFs:  95%|█████████▍| 184/194 [00:03<00:00, 41.93it/s]

✓ Found Mercedes document: 2023 Spanish Grand Prix - Infringement - Car 63 - Abnormal change of direction.pdf
✓ Found Mercedes document: 2023 Spanish Grand Prix - Infringement - Mercedes - Parc Ferme.pdf
✓ Found Mercedes document: 2023 São Paulo Grand Prix - Infringement - Car 63 - Impeding at Pit Exit.pdf


Processing PDFs: 100%|██████████| 194/194 [00:03<00:00, 51.09it/s]

✓ Found Mercedes document: 2023 United States Grand Prix - Infringement - Car 44 - Technical non-compliance (Plank).pdf
✓ Found Mercedes document: 2023 United States Grand Prix - Infringement - Car 63 - Impeding of Car 16.pdf
✓ Found Mercedes document: 2023 United States Grand Prix - Infringement - Car 63 - Leaving the track.pdf
Found 23 Mercedes documents in 2023-infridgement_profile


In [22]:
# Summary statistics
if all_mercedes_docs:
    df_mercedes = pd.DataFrame(all_mercedes_docs)
    
    print("\n" + "="*50)
    print("MERCEDES DOCUMENT EXTRACTION SUMMARY")
    print("="*50)
    
    print(f"Total Mercedes documents found: {len(all_mercedes_docs)}")
    print(f"Total text length: {sum([doc['text_length'] for doc in all_mercedes_docs]):,} characters")
    
    print("\nDocuments by year:")
    year_counts = df_mercedes['year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"  {year}: {count} documents")
    
    print("\nSample documents:")
    for _, row in df_mercedes.head().iterrows():
        print(f"  - {row['pdf_file']} ({row['text_length']} chars)")
    
    # Save summary
    df_mercedes.to_csv('mercedes_documents_summary.csv', index=False)
    print("\nSummary saved to: mercedes_documents_summary.csv")
    
else:
    print("\nNo Mercedes documents found. Please check the filtering criteria.")



MERCEDES DOCUMENT EXTRACTION SUMMARY
Total Mercedes documents found: 70
Total text length: 114,921 characters

Documents by year:
  2020-infirdgement_profile: 13 documents
  2021-infridgement_profile: 17 documents
  2022-infridgement_profile: 17 documents
  2023-infridgement_profile: 23 documents

Sample documents:
  - 2020 Austrian Grand Prix - Decision - Car 44 - alleged failure to slow for yellow flags.pdf (1438 chars)
  - 2020 Austrian Grand Prix - Decision - review of decision (document 33).pdf (1124 chars)
  - 2020 Austrian Grand Prix - Offence - Car 44 - Failure to slow for yellow flags (post review).pdf (1632 chars)
  - 2020 Austrian Grand Prix - Offence - Car 44 - incident with car 23.pdf (1418 chars)
  - 2020 Austrian Grand Prix - Offence - Car 44 - Leaving the track in turn 10.pdf (1623 chars)

Summary saved to: mercedes_documents_summary.csv


In [23]:
# Display sample of extracted text for verification
if all_mercedes_docs:
    print("\n" + "="*50)
    print("SAMPLE EXTRACTED TEXT")
    print("="*50)
    
    # Get first Mercedes document
    first_doc = all_mercedes_docs[0]
    year_path = base_path / first_doc['year']
    txt_path = year_path / first_doc['txt_file']
    
    if txt_path.exists():
        with open(txt_path, 'r', encoding='utf-8') as f:
            sample_text = f.read()
        
        print(f"Document: {first_doc['pdf_file']}")
        print(f"Year: {first_doc['year']}")
        print(f"Length: {len(sample_text)} characters")
        print("\nFirst 1000 characters:")
        print("-" * 50)
        print(sample_text[:1000] + "..." if len(sample_text) > 1000 else sample_text)



SAMPLE EXTRACTED TEXT
Document: 2020 Austrian Grand Prix - Decision - Car 44 - alleged failure to slow for yellow flags.pdf
Year: 2020-infirdgement_profile
Length: 1438 characters

First 1000 characters:
--------------------------------------------------
From The Stewards To The Team Manager, Mercedes-AMG Petronas F1 TeamDocument 33 Date 04 July 2020 Time 19:44 2020 AUSTRIAN GRAND PRIX 2 - 5 July 2020 The StewardsThe Stewards, having received a report from the Race Director, summoned (document 29) and heard from the driver and team representative, have considered the following matter and determine the following: No / Driver 44 - Lewis Hamilton Competitor Mercedes-AMG Petronas F1 Team Time 15:59 Session Qualifying Fact Alleged failure to slow for single waved yellow flags between turn 5 and 7. Offence Alleged breach of Appendix H Article 2.5.5.1.b) of the FIA International Sporting Code. Decision No further action. Reason The Stewards heard from the driver of Car 44 (Lewis Hamilton) an

## Next Steps

After running this notebook:
1. Review the extracted Mercedes documents
2. Check the sample text to ensure proper extraction
3. Proceed to consolidation and preprocessing steps
4. Begin text summarization process
